# HF

> Utilities to manage HF Hub model hosting

In [ ]:
#| default_exp hf

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
from huggingface_hub import HfApi, login, hf_hub_download, Repository

In [ ]:
#| export
def create_repository(repo_name: str, private: bool = False):
    """Create a repository on the Hugging Face Hub."""
    api = HfApi()
    api.create_repo(repo_id=repo_name, exist_ok=True, private=private)

In [ ]:
#| export
def upload_model_file(repo_name: str, file_path: str, path_in_repo: str = None):
    """Upload a model file to the Hugging Face Hub."""
    api = HfApi()
    path_in_repo = path_in_repo or file_path.split("/")[-1]
    api.upload_file(
        path_or_fileobj=file_path,
        path_in_repo=path_in_repo,
        repo_id=repo_name,
        repo_type="model",
    )

In [ ]:
#| export
def upload_readme(repo_name: str, readme_path: str):
    """Upload a README.md file to the Hugging Face Hub."""
    api = HfApi()
    api.upload_file(
        path_or_fileobj=readme_path,
        path_in_repo="README.md",  # README file should always be named README.md
        repo_id=repo_name,
        repo_type="model",
    )

In [ ]:
#| export
def download_model_file(repo_name: str, file_name: str, save_path: str):
    """Download a file from the Hugging Face Hub."""
    file_path = hf_hub_download(repo_id=repo_name, filename=file_name)
    with open(save_path, "wb") as f:
        with open(file_path, "rb") as downloaded_file:
            f.write(downloaded_file.read())

In [ ]:
#| export
def load_model(file_path: str, model_class):
    """Load a PyTorch model from a file."""
    model = model_class()
    model.load_state_dict(torch.load(file_path))
    return model

In [ ]:
#| export
def list_files_in_repo(repo_name: str):
    """List all files in a Hugging Face repository."""
    api = HfApi()
    return api.list_repo_files(repo_id=repo_name, repo_type="model")

In [ ]:
#| export
def delete_file_in_repo(repo_name: str, file_path: str):
    """Delete a file from a Hugging Face repository."""
    api = HfApi()
    api.delete_file(repo_id=repo_name, path_in_repo=file_path)

In [ ]:
#| export
def clone_repository(repo_name: str, local_path: str):
    """Clone a Hugging Face repository locally."""
    return Repository(local_dir=local_path, clone_from=f"https://huggingface.co/{repo_name}")

In [ ]:
#| export
def save_and_upload_model(repo_name: str, model, file_path: str, path_in_repo: str = None, readme_path: str = None):
    """Save a PyTorch model and upload it to Hugging Face."""
    torch.save(model.state_dict(), file_path)
    upload_model_file(repo_name, file_path, path_in_repo)
    if readme_path: upload_readme(repo_name, readme_path)

In [ ]:
#| export
def generate_readme(metadata: dict) -> str:
    """
    Generate a README.md file content with YAML front matter based on the provided metadata.
    """
    # YAML Front Matter
    yaml_front_matter = "---\n"
    yaml_front_matter += f"license: {metadata.get('license', 'apache-2.0')}\n"
    tags = metadata.get("tags", [])
    yaml_front_matter += "tags:\n"
    for tag in tags:
        yaml_front_matter += f"  - {tag}\n"
    datasets = metadata.get("datasets", [])
    yaml_front_matter += "datasets:\n"
    for dataset in datasets:
        yaml_front_matter += f"  - {dataset}\n"
    yaml_front_matter += "---\n\n"

    # Header
    readme = f"# {metadata.get('model_name', 'Model')}\n\n"
    readme += f"{metadata.get('description', 'Description not provided.')}\n\n"

    # Model Details Section
    readme += "## Model Details\n\n"
    readme += f"- **Architecture**: {metadata.get('architecture', 'Unknown')}\n"
    readme += f"- **Framework**: {metadata.get('framework', 'Unknown')}\n"
    readme += f"- **Task**: {metadata.get('task', 'Unknown')}\n"
    readme += f"- **Datasets**: {', '.join(datasets) if datasets else 'None'}\n\n"

    # Metrics Section
    metrics = metadata.get("metrics", {})
    if metrics:
        readme += "## Metrics\n\n"
        readme += "| Metric | Value |\n|--------|-------|\n"
        for metric, value in metrics.items():
            readme += f"| {metric} | {value} |\n"
        readme += "\n"
    else:
        readme += "## Metrics\n\nNo metrics provided.\n\n"

    # Usage Section
    usage_example = metadata.get("usage_example", "")
    readme += "## Usage\n\n"
    if usage_example:
        readme += f"To use this model, load it from the Hugging Face Hub:\n\n```python\n{usage_example}\n```\n\n"
    else:
        readme += "Usage example not provided.\n\n"

    # Licensing Section
    license_text = metadata.get("license", "Apache 2.0")
    readme += "## Licensing\n\n"
    readme += f"This model is shared under the {license_text} license.\n"

    return yaml_front_matter + readme

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()